In [1]:
import ee

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=NX0cR4JuYblKSA1xIG3JoK-3xvb8Bi62aMBR2-fWoqs&tc=q3NC6Xy479lnAY4r1PXbXhJHsE8lW1cKMALlNh2CW3o&cc=4jSyJSWXSNG1RNBBC0t0VKpMX7nXAg1qng9_ARy2H5M

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgXh-3-bTMU4OsQ_N3Gg14C3JjNHbGFhAuLkreovftArXQpieEJDyk

Successfully saved authorization token.


In [3]:
!pip install geemap
import geemap
Map = geemap.Map()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 13.7 MB/s 
     |████████████████████████████████| 187 kB 60.8 MB/s 
     |████████████████████████████████| 3.3 MB 55.4 MB/s 
     |████████████████████████████████| 98 kB 7.0 MB/s 
     |████████████████████████████████| 1.3 MB 30.6 MB/s 
     |████████████████████████████████| 8.8 MB 56.7 MB/s 
     |████████████████████████████████| 130 kB 54.1 MB/s 
     |████████████████████████████████| 99 kB 7.9 MB/s 
     |████████████████████████████████| 1.2 MB 25.4 MB/s 
     |████████████████████████████████| 46 kB 3.4 MB/s 
     |████████████████████████████████| 3.0 MB 33.9 MB/s 
     |████████████████████████████████| 95 kB 3.6 MB/s 
     |████████████████████████████████| 128 kB 58.3 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 342 kB 62.8 MB/s 
     |████████████████████████████████| 4

In [ ]:
# The Sentinel Dataset

def maskS2clouds(image):
  qa = image.select('QA60')
  # Bits 10 and 11 are clouds and cirrus, respectively
  cloudBitMask= 1<<10
  cirrusBitMask = 1<<11
  #setting both flags to zero to indicate clear conditions
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

#Maping the fucntion over an epoch

S2 = ee.ImageCollection("COPERNICUS/S2").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                      .filterDate('2022-01-01', '2022-03-31') \
                      .map(maskS2clouds) \
                      .median()

# function for the indices
def addVegIndices(image):
  # ndvi 
  NDVI_S2 = S2.expression('(nir-red)/(nir+red)',{
    'nir': S2.select('B8'),
    'red': S2.select('B4'),
  }).rename('NDVI_S2')

  # BSI 
  BSI_S2 = S2.expression('((swir1 + red)-(nir+blue))/ ((swir1 + red)-(nir+blue))',{
      'swir1': S2.select('B11'),
      'nir':S2.select('B8'),
      'red': S2.select('B4'),
      'blue': S2.select('B2')
  }).rename('BSI_S2')

  # Green coverage index 
  GCI_S2 = S2.expression('(water/green)-1',{
      'water':S2.select('B9'),
      'green':S2.select('B3')
  }).rename('GCI_S2')

  # GNDVI 
  GNDVI_S2 = S2.expression('(nir-green)/(nir+green)',{
      'nir':S2.select('B8'),
      'green': S2.select('B3')
  }).rename('GNDVI_S2')

  #EVI 
  EVI_S2 = S2.expression('(2.5 * ((nir- red) / (nir + 6 * red- 7.5 * blue + 1)))',{
      'nir': S2.select('B8'),
      'red': S2.select('B4'),
      'blue': S2.select('B2')
  }).rename('EVI_S2')

  # SAVI 
  SAVI_S2 = S2.expression('(nir - red) / (nir + red + 0.428) * (1.428)', {
      'nir': S2.select('B8'),
      'red': S2.select('B4')
  }).rename('SAVI_S2')

  return image.addBands(NDVI_S2).addBands(GCI_S2).addBands(GNDVI_S2).addBands(EVI_S2).addBands(SAVI_S2).addBands(BSI_S2)

# S2_visParams = {
#     'min': -22,
#     'max': 429496.75,
#     'palette': ['#11fc03', '#57f703', '#9bfa06', '#cff904', '#ecfa07']
#     }

composite_S2 = addVegIndices(S2)


bBox = ee.Geometry.BBox(36.670697, -1.041734, 36.717424, -1.087474)
# Map.addLayer(composite_S2.clip(bBox), S2_visParams, 'S2')
Map.addLayer(composite_S2.clip(bBox))
Map.setCenter(36.6940605, -1.064604, 14)
Map

Map(bottom=2109856.0, center=[-1.064604, 36.6940605], controls=(WidgetControl(options=['position', 'transparen…

<IPython.core.display.Javascript object>

In [ ]:
# calculate the standard deviation
S2_sigma = composite_S2.reduceRegion(**{
    'reducer': ee.Reducer.stdDev(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9
})

print('Std values', S2_sigma.getInfo())

# calculate the minimum value
S2_min = composite_S2.reduceRegion(**{
    'reducer': ee.Reducer.min(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9
})

print('Min values', S2_min.getInfo())

# calculate the maximum value
S2_max = composite_S2.reduceRegion(**{
    'reducer': ee.Reducer.max(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9
})

print('Max values', S2_max.getInfo())


Std values {'B1': 0.004384780858491106, 'B10': 0.00024547917766446197, 'B11': 0.03311501426733149, 'B12': 0.03216233704906506, 'B2': 0.007133671467639159, 'B3': 0.01175045735189212, 'B4': 0.018190082564076306, 'B5': 0.020608074332222514, 'B6': 0.04336817276506093, 'B7': 0.054934055108254394, 'B8': 0.056216715462400396, 'B8A': 0.058976958790337916, 'B9': 0.016397017232437427, 'BSI_S2': 0.005848253223377087, 'EVI_S2': 0.14629825758489823, 'GCI_S2': 0.09690385118514258, 'GNDVI_S2': 0.054640055545462925, 'NDVI_S2': 0.08112505311563724, 'QA10': 0, 'QA20': 0, 'QA60': 0, 'SAVI_S2': 0.07619853083354773}
Min values {'B1': 0.19140000641345978, 'B10': 0.10125000029802322, 'B11': 0.1599999964237213, 'B12': 0.12919999659061432, 'B2': 0.16680000722408295, 'B3': 0.15354999899864197, 'B4': 0.13490000367164612, 'B5': 0.15369999408721924, 'B6': 0.18440000712871552, 'B7': 0.20579999685287476, 'B8': 0.18250000476837158, 'B8A': 0.20839999616146088, 'B9': 0.16795000433921814, 'BSI_S2': 0, 'EVI_S2': 0.087069

In [ ]:
# Classify NDVI into 5 classes
ndvi_S2 = ee.Image(1) \
          .where(NDVI_S2.gt(0.0).And(NDVI_S2.lte(0.2)), 2) \
          .where(NDVI_S2.gt(0.2).And(NDVI_S2.lte(0.4)), 3) \
          .where(NDVI_S2.gt(0.4).And(NDVI_S2.lte(0.6)), 4) \
          .where(NDVI_S2.gt(0.6), 5)
ndvi_S2 = ndvi_S2.clip(bBox)
# Add map layers
Map.addLayer(ndvi_S2, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Sentinel_2 NDVI',True)

# Classify BSI into 5 classes
bsi_S2 = ee.Image(1) \
          .where(BSI_S2.gt(0.0).And(BSI_S2.lte(0.2)), 2) \
          .where(BSI_S2.gt(0.2).And(BSI_S2.lte(0.4)), 3) \
          .where(BSI_S2.gt(0.4).And(BSI_S2.lte(0.6)), 4) \
          .where(BSI_S2.gt(0.6), 5)
bsi_S2 = bsi_S2.clip(bBox)
# Add map layers
Map.addLayer(bsi_S2, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Sentinel_2 BSI',True)

# Classify GCI into 5 classes
gci_S2 = ee.Image(1) \
          .where(GCI_S2.gt(0.0).And(GCI_S2.lte(0.2)), 2) \
          .where(GCI_S2.gt(0.2).And(GCI_S2.lte(0.3)), 3) \
          .where(GCI_S2.gt(0.3).And(GCI_S2.lte(0.4)), 4) \
          .where(GCI_S2.gt(0.4), 5)
gci_S2 = gci_S2.clip(bBox)
# Add map layers
Map.addLayer(gci_S2, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Sentinel_2 GCI',True)

# Classify EVI into 5 classes
evi_S2 = ee.Image(1) \
          .where(EVI_S2.gt(0.0).And(EVI_S2.lte(0.2)), 2) \
          .where(EVI_S2.gt(0.2).And(EVI_S2.lte(0.4)), 3) \
          .where(EVI_S2.gt(0.4).And(EVI_S2.lte(0.6)), 4) \
          .where(EVI_S2.gt(0.6), 5)
evi_S2 = evi_S2.clip(bBox)
# Add map layers
Map.addLayer(evi_S2, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Sentinel_2 EVI',True)

# Classify GNDVI into 5 classes
gndvi_S2 = ee.Image(1) \
          .where(GNDVI_S2.gt(0.0).And(GNDVI_S2.lte(0.2)), 2) \
          .where(GNDVI_S2.gt(0.2).And(GNDVI_S2.lte(0.3)), 3) \
          .where(GNDVI_S2.gt(0.3).And(GNDVI_S2.lte(0.4)), 4) \
          .where(GNDVI_S2.gt(0.4), 5)
gndvi_S2 = gndvi_S2.clip(bBox)
# Add map layers
Map.addLayer(gndvi_S2, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Sentinel_2 GNDVI',True)

# Classify SAVI into 5 classes
savi_S2 = ee.Image(1) \
          .where(SAVI_S2.gt(0.0).And(SAVI_S2.lte(0.2)), 2) \
          .where(SAVI_S2.gt(0.2).And(SAVI_S2.lte(0.3)), 3) \
          .where(SAVI_S2.gt(0.3).And(SAVI_S2.lte(0.4)), 4) \
          .where(SAVI_S2.gt(0.4), 5)
savi_S2 = savi_S2.clip(bBox)
# Add map layers
Map.addLayer(savi_S2, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Sentinel_2 SAVI',True)

classes = ['0-20%', 
          '20%-40%', 
          '40%-60%', 
          '60%-80%', 
          '80%-100%']
Map

Map(bottom=2109855.0, center=[-1.0644967836973012, 36.6994857788086], controls=(WidgetControl(options=['positi…

In [ ]:
# Landsat 8 
# Import the Landsat 8 TOA image collection.
L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2022-03-01', '2022-03-31')

# Applies scaling factors.
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True) \
              .addBands(thermalBands, None, True)

def maskL8sr(image):
  # Bits 4 and 3 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = (1 << 4)
  cloudsBitMask = (1 << 3)
  cirrusBitMask = (1 << 2)
  # Get the pixel QA band.
  qa = image.select('QA_PIXEL')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                 .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask)  

L8 = L8.map(applyScaleFactors).map(maskL8sr)
L8 = L8.median()

def addlandsatIndices(image):
  # NDVI
  NDVI_L8 = L8.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI_L8')

  # BSI (Landsat 8) = (B6 + B4) – (B5 + B2) / (B6 + B4) + (B5 + B2)
  BSI_L8 = L8.expression('((swir1 + red) - (nir+blue)) / ((swir1 + red)-(nir+blue))',{
      'swir1': L8.select('SR_B6'),
      'nir':L8.select('SR_B5'),
      'red': L8.select('SR_B4'),
      'blue': L8.select('SR_B2')
  }).rename('BSI_L8')

  # GCI (Landsat 8) = (B5 / B3) -1
  GCI_L8 = L8.expression('(nir/green)-1',{
      'nir':L8.select('SR_B5'),
      'green':L8.select('SR_B3')
  }).rename('GCI_L8')

  # SAVI (Landsat 8) = ((B5 – B4) / (B5+ B4 + 0.5)) * (1.5)
  SAVI_L8 = L8.expression('(nir - red) / (nir + red + 0.5) * (1.5)', {
      'nir': L8.select('SR_B5'),
      'red': L8.select('SR_B4')
  }).rename('SAVI_L8')

  # EVI (Landsat 8) = 2.5 * ((B5 – B4) / (B5 + 6 * B4 – 7.5 * B2 + 1))
  EVI_L8 = L8.expression('(2.5 * ((nir - red) / (nir + 6 * red- 7.5 * blue + 1)))',{
      'nir': L8.select('SR_B5'),
      'red': L8.select('SR_B4'),
      'blue': L8.select('SR_B2')
  }).rename('EVI_L8')

  #GNDVI (Landsat 8) = (B5 – B3) / (B5 + B3)
  GNDVI_L8 = L8.expression('(nir - green)/(nir + green)',{
      'nir':L8.select('SR_B5'),
      'green': L8.select('SR_B3')
  }).rename('GNDVI_L8')

  return image.addBands(NDVI_L8).addBands(GCI_L8).addBands(GNDVI_L8).addBands(EVI_L8).addBands(SAVI_L8).addBands(BSI_L8)

# composite
composite_L8 = addlandsatIndices(L8)


# Display the result.
bBox = ee.Geometry.BBox(36.670697, -1.041734, 36.717424, -1.087474)

# Map.addLayer(composite_L8.clip(bBox), L8_visParams, 'L8')
Map.addLayer(composite_L8.clip(bBox))

Map.setCenter(36.777911, -1.058634, 15)
Map

Map(bottom=2109856.0, center=[-1.058634, 36.777911], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# calculate the standard deviation
L8_stdDev = composite_L8.reduceRegion(**{
    'reducer': ee.Reducer.stdDev(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9
})

print('Landsat std dev', L8_stdDev.getInfo())

# calculate the min value
L8_min = composite_L8.reduceRegion(**{
    'reducer': ee.Reducer.min(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9
})

print('Landsat min', L8_min.getInfo())

# calculate the max value
L8_max = composite_L8.reduceRegion(**{
    'reducer': ee.Reducer.max(),
    'geometry': bBox,
    'scale': 30,
    'maxPixels': 1e9
})

print('Landsat max', L8_max.getInfo())

Landsat std dev {'BSI_L8': 0, 'EVI_L8': 0.11156994034117951, 'GCI_L8': 1.5872948978544859, 'GNDVI_L8': 0.06551210415946565, 'NDVI_L8': 0.11235965218137399, 'QA_PIXEL': 0, 'QA_RADSAT': 0, 'SAVI_L8': 0.09333064638721872, 'SR_B1': 0.007271205802050154, 'SR_B2': 0.008153895459357878, 'SR_B3': 0.012731648384572953, 'SR_B4': 0.02017873497947501, 'SR_B5': 0.06108947992591837, 'SR_B6': 0.039529934996269694, 'SR_B7': 0.04118600165661511, 'SR_QA_AEROSOL': 14.230408497253103, 'ST_ATRAN': 76.37166402587275, 'ST_B10': 3.43899291586986, 'ST_CDIST': 111.38354387116733, 'ST_DRAD': 28.7403957187633, 'ST_EMIS': 29.376000058069224, 'ST_EMSD': 50.32700557481037, 'ST_QA': 13.472594741288784, 'ST_TRAD': 401.61504550747463, 'ST_URAD': 62.053784344032564}
Landsat min {'BSI_L8': 1, 'EVI_L8': 0.10026591309995006, 'GCI_L8': 0.8935586568688303, 'GNDVI_L8': 0.3088095880647415, 'NDVI_L8': 0.18431598457145718, 'QA_PIXEL': 21824, 'QA_RADSAT': 0, 'SAVI_L8': 0.11541428532070502, 'SR_B1': -0.003952499999999998, 'SR_B2':

In [ ]:
# Classify NDVI into 5 classes
ndvi_L8 = ee.Image(1) \
          .where(NDVI_L8.gt(0.0).And(NDVI_L8.lte(0.4)), 2) \
          .where(NDVI_L8.gt(0.4).And(NDVI_L8.lte(0.6)), 3) \
          .where(NDVI_L8.gt(0.6).And(NDVI_L8.lte(0.8)), 4) \
          .where(NDVI_L8.gt(0.8), 5)
ndvi_L8 = ndvi_L8.clip(bBox)
# Add map layers
Map.addLayer(ndvi_L8, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Landsat NDVI',True)

# Classify BSI into 5 classes
bsi_L8 = ee.Image(1) \
          .where(BSI_L8.gt(0.0).And(BSI_L8.lte(0.2)), 2) \
          .where(BSI_L8.gt(0.2).And(BSI_L8.lte(0.4)), 3) \
          .where(BSI_L8.gt(0.4).And(BSI_L8.lte(0.6)), 4) \
          .where(BSI_L8.gt(0.6), 5)
bsi_L8 = bsi_L8.clip(bBox)
# Add map layers
Map.addLayer(bsi_L8, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Landsat BSI',True)

# Classify GCI into 5 classes
gci_L8 = ee.Image(1) \
          .where(GCI_L8.gt(0.0).And(GCI_L8.lte(3.0)), 2) \
          .where(GCI_L8.gt(3.0).And(GCI_L8.lte(6.0)), 3) \
          .where(GCI_L8.gt(6.0).And(GCI_L8.lte(9.0)), 4) \
          .where(GCI_L8.gt(9.0), 5)
gci_L8 = gci_L8.clip(bBox)
# Add map layers
Map.addLayer(gci_L8, {'min': 0, 'max': 20, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Landsat GCI',True)

# Classify EVI into 5 classes
evi_L8 = ee.Image(1) \
          .where(EVI_L8.gt(0.0).And(EVI_L8.lte(0.2)), 2) \
          .where(EVI_L8.gt(0.2).And(EVI_L8.lte(0.4)), 3) \
          .where(EVI_L8.gt(0.4).And(EVI_L8.lte(0.6)), 4) \
          .where(EVI_L8.gt(0.6), 5)
evi_L8 = evi_L8.clip(bBox)
# Add map layers
Map.addLayer(evi_L8, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Landsat EVI',True)

# Classify GNDVI into 5 classes
gndvi_L8 = ee.Image(1) \
          .where(GNDVI_L8.gt(0.0).And(GNDVI_L8.lte(0.3)), 2) \
          .where(GNDVI_L8.gt(0.3).And(GNDVI_L8.lte(0.5)), 3) \
          .where(GNDVI_L8.gt(0.5).And(GNDVI_L8.lte(0.7)), 4) \
          .where(GNDVI_L8.gt(0.7), 5)
gndvi_L8 = gndvi_L8.clip(bBox)
# Add map layers
Map.addLayer(gndvi_L8, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Landsat GNDVI',True)

# Classify SAVI into 5 classes
savi_L8 = ee.Image(1) \
          .where(SAVI_L8.gt(0.0).And(SAVI_L8.lte(0.2)), 2) \
          .where(SAVI_L8.gt(0.2).And(SAVI_L8.lte(0.4)), 3) \
          .where(SAVI_L8.gt(0.4).And(SAVI_L8.lte(0.6)), 4) \
          .where(SAVI_L8.gt(0.6), 5)
savi_L8 = savi_L8.clip(bBox)
# Add map layers
Map.addLayer(savi_L8, {'min': 0, 'max': 5, 'palette': ['#654321','#FFA500','#FFFF00', '#00FF00', '#008000']}, 'Landsat SAVI',True)

classes = ['0-20%', 
          '20%-40%', 
          '40%-60%', 
          '60%-80%', 
          '80%-100%']
Map

Map(bottom=2109855.0, center=[-1.0644967836973012, 36.6994857788086], controls=(WidgetControl(options=['positi…